## 1.Import libraries

In [1]:
import pandas as pd
import numpy as np
import re

## 2. Load de data 

In [2]:
df = pd.read_csv("../input/attacks.csv",encoding = "ISO-8859-1")
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [3]:
df.dtypes

Case Number                object
Date                       object
Year                      float64
Type                       object
Country                    object
Area                       object
Location                   object
Activity                   object
Name                       object
Sex                        object
Age                        object
Injury                     object
Fatal (Y/N)                object
Time                       object
Species                    object
Investigator or Source     object
pdf                        object
href formula               object
href                       object
Case Number.1              object
Case Number.2              object
original order            float64
Unnamed: 22                object
Unnamed: 23                object
dtype: object

In [4]:
# How big is the dataset?
df.shape


(25723, 24)

**¿There are some nan instead of Na?** 

### 2.1. Loking for NaN 

In [5]:
#Percentage of NaN along rows
null_rows = df.isnull().sum(axis = 1).apply (lambda y: y/df.shape[1]).sort_values(ascending = False)
null_rows[:5]

# With this I have seen that there are many rows in  the data frame that do not have any data. In this way, I will eliminate 
    ## those rows that present a percentage of nulls = 1 to clean up the dataframe a bit.

12861    1.0
14368    1.0
14382    1.0
14381    1.0
14380    1.0
dtype: float64

In [6]:
# To remove this rows:
df_2 = df.dropna(axis = 0, how = "all")
df_2.shape

#The number of rows was reduced from 25723 to 8703

(8703, 24)

In [7]:
df_2.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [8]:
#Now I will check the percentage of NaN along columns
perc_null = df_2.isnull().sum().apply(lambda x: x/df_2.shape[0]).sort_values(ascending=False)

In [9]:
#Time to select a determined value to columns with high percentage of nulls and probably with posibilities to remove
perc_null[perc_null > 0.7]

#Given the high percentage of cells with NaN in these columns (Unnamed22n = 0.999885, Unnamed23 =  0.999770) i poceed to remove
    #them. In addition to these columns there are three more columns with bibliographic references that are not relevant for the 
    #task and i proceed to delet them too (pdf, href formula, href). 
    
df_3 = df_2.drop(['Unnamed: 22', 'Unnamed: 23', 'pdf', 'href formula', 'href'], axis = 1) 


In [10]:
# Case Number and Date columns cointain the same information but with different structure. I proceed to remove one of them (Case
    #number column)

df_3 = df_3.drop(['Case Number'], axis = 1)
df_3.head()

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,Case Number.1,Case Number.2,original order
0,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25,2018.06.25,6303.0
1,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18,2018.06.18,6302.0
2,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09,2018.06.09,6301.0
3,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08,2018.06.08,6300.0
4,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04,2018.06.04,6299.0


In [11]:
# to avoid problems with spaces in columns names i replace the spaces with underscore 
df_3.columns = df_3.columns.str.replace(' ', '_') 
df_3.columns

Index(['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity',
       'Name', 'Sex_', 'Age', 'Injury', 'Fatal_(Y/N)', 'Time', 'Species_',
       'Investigator_or_Source', 'Case_Number.1', 'Case_Number.2',
       'original_order'],
      dtype='object')

### 2.2. Now is time to clean the selected columns. 

To realice this work I will work column by column to make them as clean as possible.

###### Date column ====> 25-Jun-2018
Along this column i am interested only in the month, because of the year is represented in the next column and the day it is not important for my hypothesis

In [12]:
# con esto saco solo los meses de toda la información que tengo en esta columna

df_3['Date'] = df_3['Date'].astype(str)
df_3["Date"] = df_3[df_3.columns[0]].apply(lambda x: re.findall(r'\b[A-Z]\w{2}\b', x))  

#Para quitar los corchetes de los valores:
df_3['Date'] = df_3['Date'].str.get(0)
df_3


,Date,Year,Type,Country,Area,Location,Activity,Name,Sex_,Age,Injury,Fatal_(Y/N),Time,Species_,Investigator_or_Source,Case_Number.1,Case_Number.2,original_order
0,Jun,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25,2018.06.25,6303.0
1,Jun,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18,2018.06.18,6302.0
2,Jun,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09,2018.06.09,6301.0
3,Jun,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08,2018.06.08,6300.0
4,Jun,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04,2018.06.04,6299.0
5,Jun,2018.0,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,NaN,"No injury, board bitten",N,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b,2018.06.03.b,6298.0
6,Jun,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a,2018.06.03.a,6297.0
7,May,2018.0,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,52,Minor injury to foot. PROVOKED INCIDENT,N,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27,2018.05.27,6296.0
8,May,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b,2018.05.26.b,6295.0
9,May,2018.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",Standing,male,M,12,Minor injury to foot,N,14h00,NaN,"K. McMurray, Tracking Sharks.com",2018.05.26.a,2018.05.26.a,6294.0


###### Year column ====> 2018.0
The objective in this column is to keep only the year eliminating .0

In [13]:
print(type(df_3.loc[df_3.index[0], 'Year']))

<class 'numpy.float64'>


In [14]:
df_3['Year'] = df_3['Year'].replace('.0', ' ')
df_3.sample(10)

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex_,Age,Injury,Fatal_(Y/N),Time,Species_,Investigator_or_Source,Case_Number.1,Case_Number.2,original_order
4265,NaN,1956.0,Unprovoked,PAPUA NEW GUINEA,Madang Province,"Singour, 60 miles south of Madang",Diving,native boy,M,NaN,Lower leg & foot lacerated,N,NaN,NaN,H.D. Baldridge,1956.00.00.h,1956.00.00.h,2038.0
5694,Jan,1891.0,Unprovoked,JAMAICA,Kingston Parish,Port Royal Harbour,boat capsized,3 sailors,M,NaN,FATAL,Y,Night,NaN,"Tuapeka Times, 1/21/1891",1891.01.08.R,1891.01.08.R,609.0
6931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1286,Mar,2008.0,Unprovoked,USA,California,"Huntington Beach, Orange County",Surfing,Thomas Larkin,M,27,"No injury to surfer, surfboard bitten by the s...",N,08h00,White shark,R. Collier,2008.03.07,2008.03.07,5017.0
708,Jan,2013.0,Invalid,AUSTRALIA,Queensland,Bullcock Beach,Dragging stranded shark into deeper water,Paul Marshallsea,M,62,"No injury, a 3 m blue shark merely snapped at ...",NaN,NaN,NaN,"The Guardian, 1/21/2013",2013.01.21.R,2013.01.21.R,5595.0
7114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3050,Nov,1980.0,Unprovoked,REUNION,Grand'Anse,Petite-Ile,Spearfishing,Jean-Michel Pothin,M,22,Bitten twice,N,15h00,1.8 m [6'] white shark,G. Van Grevelynghe,1980.11.24,1980.11.24,3253.0
3758,Mar,1963.0,Sea Disaster,SOLOMON ISLANDS,Bougainville (North Solomons),NaN,The 500-ton coastal trader Polurrian foundered,"Daniel Viva, missionary",M,NaN,FATAL,Y,Night,NaN,"Sydney Morning Herald, 4/6/1963",1963.03.30,1963.03.30,2545.0
6707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


###### Type column 
This column is OK

###### Country column
- There are countries and continents. To unify the contents I will work with continents

In [15]:
#How many unique countries are in the dataset? 
df_3["Country"].unique()

# First of all, unify the lowercase and uppercase letters:
df_3["Country"] = df_3["Country"].str.lower()

In [16]:
# Now I will create 5 varibles to definy the 5 continents

south_america = ["brazil", "ecuador", "chile"]
central_america = ["mexico","costa rica", "bahamas", "cuba", "dominican republic", "cayman islands", "aruba", "puerto rico","st. martin", "trinidad & tobago", "jamaica", "belize", "antigua", "turks & caicos", "panama"]
north_america = ["usa", "columbia" , "canada"]
africa = ["south africa", "libya", "mauritius", "egipt", "st helena, british overseas territory", "cape verde", "mozambique", "seychelles",  "nigeria", "kenya", "madagascar"]
asia = ["thailand", "maldives", "malaysia", "japan", "united arab emirates", "philippines", "indonesia", "china", "israel","taiwan", "palestinian territories", "saudi arabia", "united arab emirates (uae)",  "south korea", "vietnam"]
oceania = ["australia", "new caledonia", "new zealand", "samoa", "solomon islands", "french polynesia", "fiji", "papua new guinea", "kiribati", "guam", "tonga"]
europe = ["england", "spain","united kingdom", "italy", "greece", "france","scotland", "croatia", "russia", "azores", "malta"]

df_3['Country'] = df_3.Country.apply(lambda x: "SA" if x in south_america else x)
df_3['Country'] = df_3.Country.apply(lambda x: "CA" if x in central_america else x)
df_3['Country'] = df_3.Country.apply(lambda x: "NA" if x in north_america else x)
df_3['Country'] = df_3.Country.apply(lambda x: "Af" if x in africa else x)
df_3['Country'] = df_3.Country.apply(lambda x: "As" if x in asia else x)
df_3['Country'] = df_3.Country.apply(lambda x: "O" if x in oceania else x)
df_3['Country'] = df_3.Country.apply(lambda x: "E" if x in europe else x)

df_3

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex_,Age,Injury,Fatal_(Y/N),Time,Species_,Investigator_or_Source,Case_Number.1,Case_Number.2,original_order
0,Jun,2018.0,Boating,NA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25,2018.06.25,6303.0
1,Jun,2018.0,Unprovoked,NA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18,2018.06.18,6302.0
2,Jun,2018.0,Invalid,NA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09,2018.06.09,6301.0
3,Jun,2018.0,Unprovoked,O,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08,2018.06.08,6300.0
4,Jun,2018.0,Provoked,CA,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04,2018.06.04,6299.0
5,Jun,2018.0,Unprovoked,O,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,NaN,"No injury, board bitten",N,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b,2018.06.03.b,6298.0
6,Jun,2018.0,Unprovoked,SA,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a,2018.06.03.a,6297.0
7,May,2018.0,Unprovoked,NA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,52,Minor injury to foot. PROVOKED INCIDENT,N,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27,2018.05.27,6296.0
8,May,2018.0,Unprovoked,NA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b,2018.05.26.b,6295.0
9,May,2018.0,Unprovoked,NA,Florida,"Daytona Beach, Volusia County",Standing,male,M,12,Minor injury to foot,N,14h00,NaN,"K. McMurray, Tracking Sharks.com",2018.05.26.a,2018.05.26.a,6294.0


###### Sex column 0.34 null values
I this case I try to reemplace the NaN data for a random selectinon of sex. We have different unique values: 

- 'F'
- 'M'
- **nan**
- **'M '**
- **'lli'**
- **'N'**
- **'.'**

marked in bld the characters that I proceed to remove

In [28]:
import random
fill_list = ["M", "F"]
df_3["Sex_"] = df_3["Sex_"].fillna(pd.Series(np.random.choice(fill_list)))


In [18]:
df_3.replace({"Sex_": {"nan": " ", "lli": " ", "N": " ", ".": " " }})

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex_,Age,Injury,Fatal_(Y/N),Time,Species_,Investigator_or_Source,Case_Number.1,Case_Number.2,original_order
0,Jun,2018.0,Boating,NA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25,2018.06.25,6303.0
1,Jun,2018.0,Unprovoked,NA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18,2018.06.18,6302.0
2,Jun,2018.0,Invalid,NA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09,2018.06.09,6301.0
3,Jun,2018.0,Unprovoked,O,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08,2018.06.08,6300.0
4,Jun,2018.0,Provoked,CA,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04,2018.06.04,6299.0
5,Jun,2018.0,Unprovoked,O,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,NaN,"No injury, board bitten",N,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b,2018.06.03.b,6298.0
6,Jun,2018.0,Unprovoked,SA,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a,2018.06.03.a,6297.0
7,May,2018.0,Unprovoked,NA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,52,Minor injury to foot. PROVOKED INCIDENT,N,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27,2018.05.27,6296.0
8,May,2018.0,Unprovoked,NA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b,2018.05.26.b,6295.0
9,May,2018.0,Unprovoked,NA,Florida,"Daytona Beach, Volusia County",Standing,male,M,12,Minor injury to foot,N,14h00,NaN,"K. McMurray, Tracking Sharks.com",2018.05.26.a,2018.05.26.a,6294.0


In [27]:
df_3["Sex_"].unique()

array(['F', 'M', nan, 'M ', 'lli', 'N', '.'], dtype=object)

In [58]:
def replace_sex(col):
    chars =  'M ', 'lli', 'N', '.', 'nan'
    for c in chars:
        col = col.replace(c, random.choice(fill_list))
    return(col)
   

In [59]:
df_3["Sex_"] = df_3["Sex_"].apply(replace_sex)
df_3["Sex_"].unique()

array(['F', 'M'], dtype=object)

###### Age
In this case I replace the NaN for the mean values of the mean. First of all  I try to clean all the data. We have several types of error in this column

- Ages in string format
- Ages separated by *&*, *or*, *to*, *>*
- Ages with *?*

In [20]:
df_3['Age'] = df_3['Age'].astype(str) # To work with re.findall function we need string not integer. 


pattern = "\d+"  #to remove those values with strings or simbols
df_3['Age'] = df_3['Age'].apply(lambda x: re.findall(pattern, x))
df_3

#but now we have some cells with multiple values. I asume that in this attack, one shark attacked many people, hence the  
    #different ages in the same cell
df_3.explode('Age')

df_3['Age'] = df_3['Age'].str.get(0)
df_3


,Date,Year,Type,Country,Area,Location,Activity,Name,Sex_,Age,Injury,Fatal_(Y/N),Time,Species_,Investigator_or_Source,Case_Number.1,Case_Number.2,original_order
0,Jun,2018.0,Boating,NA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25,2018.06.25,6303.0
1,Jun,2018.0,Unprovoked,NA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18,2018.06.18,6302.0
2,Jun,2018.0,Invalid,NA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09,2018.06.09,6301.0
3,Jun,2018.0,Unprovoked,O,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08,2018.06.08,6300.0
4,Jun,2018.0,Provoked,CA,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04,2018.06.04,6299.0
5,Jun,2018.0,Unprovoked,O,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,NaN,"No injury, board bitten",N,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b,2018.06.03.b,6298.0
6,Jun,2018.0,Unprovoked,SA,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a,2018.06.03.a,6297.0
7,May,2018.0,Unprovoked,NA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,52,Minor injury to foot. PROVOKED INCIDENT,N,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27,2018.05.27,6296.0
8,May,2018.0,Unprovoked,NA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b,2018.05.26.b,6295.0
9,May,2018.0,Unprovoked,NA,Florida,"Daytona Beach, Volusia County",Standing,male,M,12,Minor injury to foot,N,14h00,NaN,"K. McMurray, Tracking Sharks.com",2018.05.26.a,2018.05.26.a,6294.0


Once I have the column of clean years I process to replace the NaN. in this case, I will substitute the NaN for the mean of this column.

In [21]:
## first of all we need to convert str to int
df_3["Age"] = df_3["Age"].fillna(-1)
df_3["Age"] = df_3["Age"].astype(int)
df_3["Age"] = df_3["Age"].replace(-1, np.nan)

In [22]:
def mean_age(col):
    
    mean_age = col.mean()
    return mean_age

In [23]:
value_to_replace = mean_age(df_3['Age'])
value_to_replace = round(value_to_replace, 2)
value_to_replace

27.32

In [25]:
df_3["Age"] = df_3["Age"].fillna(value_to_replace)

###### Time

In [70]:
df_3['Time'] = df_3['Time'].str.strip()
df_3['Time'] = df_3['Time'].str.lower()
df_3.loc[df_3['Time'].notnull(),'Time'].sample(20)

1873           12h00
406            11h41
1752           08h00
1962           07h00
271            16h00
3732           11h40
375            12h30
2793           14h00
295            11h00
1953           11h00
3571           07h00
1638           09h00
2299       afternoon
4008           12h00
954            12h30
4088            a.m.
5136           11h30
1157    before 07h00
426            15h00
2752           06h30
Name: Time, dtype: object

In [ ]:
attacks.loc[(attacks['Time'].notnull()) & (attacks['Time'].str.fullmatch('[a-zA-Z]{3,}')),'Time'].value_counts()

In [72]:
df_3['Time'].replace(['.*evening.*','.*dusk.*',
                         '.*dark.*','.*night.*'],'night',regex=True,inplace=True)

df_3['Time'].replace(['pm','p\.m\.','.*midday.*','.*sunset.*','.*lunchtime.*',
                         '.*noon.*','.*daytime.*','.*afternoon.*'],'afternoon',regex=True,inplace=True)

df_3['Time'].replace(['a\.m\.''am','.*dawn.*',
                         '.*daybreak.*','.*morning.*'],'day-morning',regex=True,inplace=True)

In [75]:
#grouping 06:00 to 11:59 as day-morning because english is weird
df_3['Time'].replace(to_replace='.*0[6-9].?\d{2}.*',value='day-morning',regex=True,inplace=True)
df_3['Time'].replace(to_replace='.*1[0-1].?\d{2}.*',value='day-morning',regex=True,inplace=True)
#grouping 12:00 to 17:59 as afternoon
df_3['Time'].replace(to_replace='.*1[2-7].?\d{2}.*',value='afternoon',regex=True,inplace=True)
#grouping 18:00 to 23:59 as night
df_3['Time'].replace(to_replace='.*1[8-9].?\d{2}.*',value='night',regex=True,inplace=True)
df_3['Time'].replace(to_replace='.*2[0-3].?\d{2}.*',value='night',regex=True,inplace=True)
#grouping 00:00 to 05:59 as night-morning because english is weird
df_3['Time'].replace(to_replace='.*0[0-5].?\d{2}.*',value='night-morning',regex=True,inplace=True)

In [77]:
df_3[~df_3['Time'].isin(['day-morning','night-morning','night','afternoon'])]['Time'].count()

32

In [78]:
df_3.loc[~df_3['Time'].isin(['day-morning','night-morning','night','afternoon']),'Time'] = np.nan

In [79]:
df_3['Time'].value_counts(dropna=False)

NaN              5787
afternoon        1593
day-morning       923
night             359
night-morning      41
Name: Time, dtype: int64

###### Species

In [66]:
df_3["Species_"].unique()

array(['White shark', nan, '2 m shark', ..., "12' tiger shark",
       'Blue pointers',
       'Said to involve a grey nurse shark that leapt out of the water and  seized the boy but species identification is questionable'],
      dtype=object)

In [68]:
df_3["Species_"].value_counts(dropna=False)

NaN                                                               5239
White shark                                                        163
Shark involvement prior to death was not confirmed                 105
Invalid                                                            102
Shark involvement not confirmed                                     88
                                                                  ... 
76 cm [2.5'] carpet shark                                            1
2 days later a 600-lb shark was caught 100 yards from the site       1
Tiger shark, 3.7 m to 4.3 m  [12' to 14']                            1
9.5' shark?                                                          1
White shark, 5' to 6' juvenile                                       1
Name: Species_, Length: 1550, dtype: int64